# Are the Final Scores of WNBA Teams Mostly Even or Odd?

## I once came across a claim stating that the majority of scores for sports teams are odd, and I found myself questioning its validity. Curious to explore this further, I decided to conduct my own analysis to determine whether this assertion holds true across different teams and games. Through this project, I aimed to uncover patterns in scoring and understand how these dynamics vary among various teams.



In [18]:
import pandas as pd
from requests_html import AsyncHTMLSession
import nest_asyncio

# Apply nest_asyncio to avoid 'event loop already running' error
nest_asyncio.apply()

# Set the URL
url = "https://www.livesport.com/basketball/usa/wnba/results/"

# Create an asynchronous session
asession = AsyncHTMLSession()

async def get_scores():
    # Fetch the page
    response = await asession.get(url)
    
    # Render the JavaScript to get the dynamic content with extended wait time
    await response.html.arender(wait=30)

    # Use the correct CSS selectors to find the scores and team names
    home_teams = response.html.find('.event__participant--home')
    away_teams = response.html.find('.event__participant--away')
    home_scores = response.html.find('.event__score--home')
    away_scores = response.html.find('.event__score--away')
    
    # Initialize a dictionary to hold team names and their scores
    team_scores = {}

    # Extract team names and scores, mapping each team to a list of scores
    for i in range(len(home_teams)):
        home_team = home_teams[i].text.strip().replace('@', '')
        away_team = away_teams[i].text.strip().replace('@', '')
        
        # Remove " W" only from the end of the team names
        home_team = home_team.rstrip(' W')
        away_team = away_team.rstrip(' W')
        
        # Ensure that scores are being extracted correctly
        home_score = int(home_scores[i].text.strip())
        away_score = int(away_scores[i].text.strip())

        # Add home team score
        if home_team not in team_scores:
            team_scores[home_team] = []
        team_scores[home_team].append(home_score)

        # Add away team score
        if away_team not in team_scores:
            team_scores[away_team] = []
        team_scores[away_team].append(away_score)

    return team_scores  # Return the dictionary directly

# Run the asynchronous function and get the results
scores = asession.run(get_scores)

# Check the type and content of scores
if isinstance(scores, list) and len(scores) == 1:
    scores = scores[0]  # Extract the dictionary from the list

# Check the type and content of the scores variable
print("Type of Scores:", type(scores))  # Should be <class 'dict'>
print("Scores Dictionary:", scores)  # This should print a dictionary

# Now, calculate even/odd percentages
even_odd_percentages = []

if isinstance(scores, dict) and scores:
    for team, scores_list in scores.items():
        even_count = sum(1 for score in scores_list if score % 2 == 0)
        odd_count = sum(1 for score in scores_list if score % 2 != 0)
        total_games = len(scores_list)
        
        even_percentage = (even_count / total_games) * 100 if total_games > 0 else 0
        odd_percentage = (odd_count / total_games) * 100 if total_games > 0 else 0
        
        even_odd_percentages.append({
            'Team': team,
            'Even Percentage': even_percentage,
            'Odd Percentage': odd_percentage
        })

# Create a DataFrame for even/odd percentages
percent_df = pd.DataFrame(even_odd_percentages)

# Check the percent_df content
print("Even/Odd Percentages DataFrame:", percent_df)

Type of Scores: <class 'dict'>
Scores Dictionary: {'Dallas Wings': [84, 109, 81, 67, 91, 77, 96, 86, 93, 94, 93, 113, 71, 74, 91, 101, 81], 'Las Vegas Aces': [98, 85, 84, 78, 86, 71, 72, 90, 97, 83, 90, 77, 74, 87, 87, 67, 85, 89], 'Seattle Storm': [89, 72, 90, 83, 90, 90, 70, 71, 86, 85, 85, 72, 83, 75, 81, 89, 81], 'Phoenix Mercury': [70, 85, 93, 69, 66, 77, 74, 79, 76, 70, 82, 63, 86, 89, 85, 96, 69], 'Los Angeles Sparks': [68, 81, 87, 82, 66, 67, 78, 86, 62, 94, 110, 74, 61, 71, 86, 68, 83, 87], 'Minnesota Lynx': [51, 78, 88, 83, 76, 78, 99, 79, 76, 89, 90, 87, 98, 99, 79, 86, 74], 'Atlanta Dream': [78, 86, 76, 69, 64, 100, 107, 66, 80, 72, 81, 79, 80, 72, 82, 83, 79, 70], 'New York Liberty': [67, 87, 79, 99, 105, 75, 77, 98, 88, 84, 64, 79, 94, 79, 103, 82, 81], 'Chicago Sky': [54, 70, 88, 66, 58, 92, 92, 71, 74, 81, 70, 75, 80, 68, 90, 65, 93, 67], 'Connecticut Sun': [87, 76, 71, 88, 86, 79, 67, 64, 93, 96, 80, 72, 82, 69, 70, 109, 74, 96], 'Indiana Fever': [91, 110, 74, 75, 104,

In [19]:
display(percent_df)

,Team,Even Percentage,Odd Percentage
0,Dallas Wings,29.411765,70.588235
1,Las Vegas Aces,44.444444,55.555556
2,Seattle Storm,41.176471,58.823529
3,Phoenix Mercury,47.058824,52.941176
4,Los Angeles Sparks,61.111111,38.888889
5,Minnesota Lynx,52.941176,47.058824
6,Atlanta Dream,66.666667,33.333333
7,New York Liberty,35.294118,64.705882
8,Chicago Sky,66.666667,33.333333
9,Connecticut Sun,61.111111,38.888889


In conclusion, the initial theory regarding even and odd scoring percentages proved to be incorrect, as results varied significantly from team to team. For instance, teams like the LA Sparks and Indiana Fever exhibited notably higher percentages of odd scores, reflecting their younger rosters and propensity for late-game comebacks. In contrast, more established teams, such as the Las Vegas Aces, showcased a more balanced scoring pattern. This discrepancy could indicate that younger teams are still developing their strategies, leading to unpredictable scoring, while veteran teams may have more consistent scoring behaviors. These findings suggest that a team’s experience and style of play can greatly influence their scoring dynamics.